In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
import re

import numpy as np
import pandas as pd

import json

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

from collections import Counter

In [90]:
data = pd.read_csv("/content/drive/MyDrive/Project/[SJU]창의학기제/lecture_evaluation_2021/data/review_datal_all.csv")

In [4]:
!pip install kss

     |████████████████████████████████| 71kB 2.0MB/s 


In [100]:
import kss #문장 split
data["sentence"] = data["text"].apply(kss.split_sentences)

In [43]:
def split_sen(data): #.(공백)을 기준으로 문장 split
  result = []
  for list_sentence in data:
    list_sentence = str(list_sentence).split('. ')
    result.append(list_sentence)
  return result
data.sentence = data.sentence.apply(split_sen)

- 자료구조 : [[문장1],[문장2],[문장3],...,[문장n]

# decide aspect
- 교수 교수님 
- 과제
- 수업 강의 강의력
- 시험 중간고사 중간 기말고사 기말 문제
- 학점 성적

In [8]:
#aspect
professor = ["교수", "교수님"]
task = ["팀플","과제","팀","플"]
test = ["시험","중간고사","중간","기말","기말고사"]
gpa = ["학점","성적"]
teaching = ["강의","강의력","수업"]

In [59]:
#data형태 : [[문장1], [문장2], [문장3]]
def find_aspect(data,df):
  for idx in range(len(data)):
    url_id = data.url_id[idx]
    for sentence in data.sentence[idx]: #한문장씩 aspect를 확인하고 딕셔너리 형태로 행을 만듦
      #한문장에 두개의 aspect가 들어가 있는 경우가 있어 if로만 구성
      if any(format in str(sentence) for format in professor): #리스트 원소 값들이 문자열에 들어있는지 확인
        df = df.append({"url_id":url_id, "sentence":str(sentence),"aspect":"professor"},ignore_index=True)
      if any(format in str(sentence) for format in task):
        df = df.append({"url_id":url_id, "sentence":str(sentence),"aspect":"task"},ignore_index=True)
      if any(format in str(sentence) for format in test):
        df = df.append({"url_id":url_id, "sentence":str(sentence),"aspect":"test"},ignore_index=True)
      if any(format in str(sentence) for format in gpa):
        df = df.append({"url_id":url_id, "sentence":str(sentence),"aspect":"gpa"},ignore_index=True)
      if any(format in str(sentence) for format in teaching):
        df = df.append({"url_id":url_id, "sentence":str(sentence),"aspect":"teaching"},ignore_index=True)
  return df

In [45]:
data.sentence[0]

[['판례평석 잘하면 성적잘 주시는것 같음.'],
 ['중간 개망치고 기말도 틀린 문제 몇 개 있었는데 판례평석 과제 평가는 전부 A+이었음.'],
 ['성적도 결국 A+ 나옴.']]

In [83]:
df = pd.DataFrame(columns=["url_id","sentence","aspect"])

In [84]:
df = find_aspect(data, df)
df

,url_id,sentence,aspect
0,535020,['판례평석 잘하면 성적잘 주시는것 같음.'],gpa
1,535020,['중간 개망치고 기말도 틀린 문제 몇 개 있었는데 판례평석 과제 평가는 전부 A+...,task
2,535020,['중간 개망치고 기말도 틀린 문제 몇 개 있었는데 판례평석 과제 평가는 전부 A+...,test
3,535020,['성적도 결국 A+ 나옴.'],gpa
4,535020,['과제들 되게 귀찮고 많다 시험은 말빨 좋으면 된다 난 시험은 망쳤지만 과제는 열...,task
...,...,...,...
18385,1703418,['교수님이 해박하시고 곤충의 장점에 대해 많이 알고 갑니다.'],professor
18386,1703418,['생각보다 공부하는 사람이 몇 없는 것 같아서 공부만 했다면 성적은 수월하게 가져...,gpa
18387,1703418,['팀플은 한 번 있고 점수에 크게 반영되지 않아서 어느 정도만 하면 될 듯합니다'...,task
18388,1703418,['팀플은 한 번 있고 점수에 크게 반영되지 않아서 어느 정도만 하면 될 듯합니다'...,test


In [85]:
extraction_aspect = pd.DataFrame(df)
extraction_aspect.head()

,url_id,sentence,aspect
0,535020,['판례평석 잘하면 성적잘 주시는것 같음.'],gpa
1,535020,['중간 개망치고 기말도 틀린 문제 몇 개 있었는데 판례평석 과제 평가는 전부 A+...,task
2,535020,['중간 개망치고 기말도 틀린 문제 몇 개 있었는데 판례평석 과제 평가는 전부 A+...,test
3,535020,['성적도 결국 A+ 나옴.'],gpa
4,535020,['과제들 되게 귀찮고 많다 시험은 말빨 좋으면 된다 난 시험은 망쳤지만 과제는 열...,task


In [89]:
extraction_aspect.to_csv("/content/drive/MyDrive/Project/[SJU]창의학기제/lecture_evaluation_2021/data/aspect_extraction.csv")